Start with a simple multi-stage autoencoder

https://blog.keras.io/building-autoencoders-in-keras.html

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import nn
import keras
from keras import layers
from keras.datasets import mnist


In [ ]:

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)


(60000, 784)
(10000, 784)


In [ ]:
def create_model(activation_forward, activation_reverse):
    input_img = keras.Input(shape=(784,))
    encoded = layers.Dense(128, activation=activation_forward)(input_img)
    encoded = layers.Dense(64, activation=activation_forward)(encoded)
    encoded = layers.Dense(32, activation=activation_forward)(encoded)

    decoded = layers.Dense(64, activation=activation_reverse)(encoded)
    decoded = layers.Dense(128, activation=activation_reverse)(decoded)
    decoded = layers.Dense(784, activation='sigmoid')(decoded)

    autoencoder = keras.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.summary()
    return autoencoder


In [ ]:
activation_forward = 'relu'
num_epochs = 1
autoencoder = create_model(activation_forward, 'relu')

autoencoder.fit(x_train, x_train,
                epochs=num_epochs,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))


235/235 [==============================] - 9s 32ms/step - loss: 0.2436 - val_loss: 0.1655


In [ ]:
activation_forward = 'sigmoid'
num_epochs = 1
autoencoder = create_model(activation_forward, 'sigmoid')

autoencoder.fit(x_train, x_train,
                epochs=num_epochs,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))


235/235 [==============================] - 9s 32ms/step - loss: 0.2907 - val_loss: 0.2632


In [ ]:
activation_forward = 'linear'
num_epochs = 1
autoencoder = create_model('linear', 'linear')

autoencoder.fit(x_train, x_train,
                epochs=num_epochs,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))


235/235 [==============================] - 6s 20ms/step - loss: 0.2043 - val_loss: 0.1346


In [ ]:
class InvertedDense(layers.Dense):
    def __init__(self, layer_sizes, *args, **kwargs):
        self.layer_sizes = layer_sizes
        self.kernels = []
        self.biases = []
        self.biases2 = []
        self.uses_learning_phase = True
        self.activation = kwargs['activation']
        super().__init__(units=1, *args, **kwargs)  # 'units' not used

    def compute_output_shape(self, input_shape):
        return input_shape

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = int(input_shape[-1])

        self.input_spec = layers.InputSpec(min_ndim=2, axes={-1: input_dim})
        # print(input_dim)
        for i in range(len(self.layer_sizes)):

            self.kernels.append(
                self.add_weight(
                    shape=(
                        input_dim,
                        self.layer_sizes[i]),
                    initializer=self.kernel_initializer,
                    name='ae_kernel_{}'.format(i),
                    regularizer=self.kernel_regularizer,
                    constraint=self.kernel_constraint))

            if self.use_bias:
                self.biases.append(
                    self.add_weight(
                        shape=(
                            self.layer_sizes[i],
                        ),
                        initializer=self.bias_initializer,
                        name='ae_bias_{}'.format(i),
                        regularizer=self.bias_regularizer,
                        constraint=self.bias_constraint))
            input_dim = self.layer_sizes[i]

        if self.use_bias:
            for n, i in enumerate(range(len(self.layer_sizes)-2, -1, -1)):
                self.biases2.append(
                    self.add_weight(
                        shape=(
                            self.layer_sizes[i],
                        ),
                        initializer=self.bias_initializer,
                        name='ae_bias2_{}'.format(n),
                        regularizer=self.bias_regularizer,
                        constraint=self.bias_constraint))
            self.biases2.append(self.add_weight(
                shape=(
                    int(input_shape[-1]),
                ),
                initializer=self.bias_initializer,
                name='ae_bias2_{}'.format(len(self.layer_sizes)),
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint))

        self.built = True

    def call(self, inputs):
        return self.decode(self.encode(inputs))

    def _apply_dropout(self, inputs):
        dropped = keras.backend.dropout(inputs, self.dropout)
        return keras.backend.in_train_phase(dropped, inputs)

    def encode(self, inputs):
        latent = inputs
        for i in range(len(self.layer_sizes)):
            if self.dropout > 0:
                latent = self._apply_dropout(latent)
            print(self.kernels[i])
            latent = keras.backend.dot(latent, self.kernels[i])
            if self.use_bias:
                print(self.biases[i])
                latent = keras.backend.bias_add(latent, self.biases[i])
            if self.activation is not None:
                latent = self.activation(latent)
        if self.l2_normalize:
            latent = latent / keras.backend.l2_normalize(latent, axis=-1)
        return latent

    def decode(self, latent):
        recon = latent
        for i in range(len(self.layer_sizes)):
            if self.dropout > 0:
                recon = self._apply_dropout(recon)
            print(self.kernels[len(self.layer_sizes) - i - 1])
            recon = keras.backend.dot(recon, K.backend.transpose(
                self.kernels[len(self.layer_sizes) - i - 1]))
            if self.use_bias:
                print(self.biases2[i])
                recon = keras.backend.bias_add(recon, self.biases2[i])
            if self.activation is not None:
                recon = self.activation(recon)
        return recon

    def get_config(self):
        config = {
            'layer_sizes': self.layer_sizes
        }
        base_config = super().get_config()
        base_config.pop('units', None)
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
# https://github.com/nanopony/keras-convautoencoder/blob/master/autoencoder_layers.py
class DependentDense(layers.Dense):
    def __init__(self, output_dim, master_layer, init='glorot_uniform', activation='linear', weights=None,
                 W_regularizer=None, b_regularizer=None, activity_regularizer=None,
                 W_constraint=None, b_constraint=None, input_dim=None, **kwargs):
        self.master_layer = master_layer
        super(DependentDense,self).__init__(output_dim, **kwargs)

    def build(self):
        self.W = self.master_layer.W.T
        self.b = K.zeros((self.output_dim,))
        self.params = [self.b]
        self.regularizers = []
        if self.W_regularizer:
            self.W_regularizer.set_param(self.W)
            self.regularizers.append(self.W_regularizer)

        if self.b_regularizer:
            self.b_regularizer.set_param(self.b)
            self.regularizers.append(self.b_regularizer)

        if self.activity_regularizer:
            self.activity_regularizer.set_layer(self)
            self.regularizers.append(self.activity_regularizer)

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights



In [ ]:
class DependentBias(layers.Layer):
    def __init__(self, master_layer, **kwargs):
        self.master_layer = master_layer
        super(DependentBias,self).__init__(**kwargs)

    def call(self, inputs):
        out = inputs - self.master_layer.b
        print('call: out', out)
        return out


In [ ]:
def linear(x):
  return x

def sigmoid(x):
    x = x + 0.000001
    output = tf.math.sigmoid(x)
    return output

def log_sigmoid(x):
    x = x + 0.000001
    output = tf.math.log_sigmoid(x)
    return output

activations = { 'sigmoid': sigmoid, 'log_sigmoid': log_sigmoid, 'linear': linear}
inverses = { 'sigmoid': 'log_sigmoid', 'linear': 'linear'}

In [ ]:
# for act, func in activations:
#     pass

In [ ]:
class TiedDense(layers.Layer):
    global activations, inverses
    def __init__(self, master_layer):
        super(TiedDense, self).__init__()
        self.master_layer = master_layer

    def build(self, input_shape):  # Create the state of the layer (weights)
        
        # self.W = self.master_layer._trainable_weights[0]
        # self.b = self.master_layer._trainable_weights[1]
        activation = self.master_layer.get_config()['activation']
        self.activation = inverses[activation]
        print('activation', self.activation)
        # self.w = tf.transpose(self.W)
        # self.trainable = False
        
    def call(self, inputs):  # Defines the computation from inputs to outputs
        W = self.master_layer._trainable_weights[0]
        b = self.master_layer._trainable_weights[1]
        w = tf.transpose(W)
        return tf.matmul((activations[self.activation](inputs) - b), w)
    

In [ ]:
def create_palindromic_model(activation_forward):
    input_img = keras.Input(shape=(784,))
    a = layers.Dense(128, activation=activation_forward)
    encoded = a(input_img)
    b = layers.Dense(64, activation=activation_forward)
    encoded = b(encoded)
    c = layers.Dense(32, activation=activation_forward)
    encoded = c(encoded)

    decoded = TiedDense(c)(encoded)
    decoded = TiedDense(b)(decoded)
    decoded = TiedDense(a)(decoded)

    autoencoder = keras.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.summary()
    autoencoder.layers[-1].trainable = False
    autoencoder.layers[-2].trainable = False
    autoencoder.layers[-3].trainable = False
    return autoencoder


In [ ]:
activation_forward = 'sigmoid'
num_epochs = 10
autoencoder = create_palindromic_model(activation_forward)

autoencoder.fit(x_train, x_train,
                epochs=num_epochs,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))


activation log_sigmoid
activation log_sigmoid
activation log_sigmoid
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 784)]             0         
                                                                 
 dense_39 (Dense)            (None, 128)               100480    
                                                                 
 dense_40 (Dense)            (None, 64)                8256      
                                                                 
 dense_41 (Dense)            (None, 32)                2080      
                                                                 
 tied_dense_15 (TiedDense)   (None, 64)                2080      
                                                                 
 tied_dense_16 (TiedDense)   (None, 128)               8256      
                                                        

In [ ]:
print(autoencoder.layers[-1]._trainable_weights)

[]
